In [1]:
# Loading Required Packages 
import numpy as np
import cobra
import networkx as nx 
import xlsxwriter 
import itertools
# Loading the Model 
model = cobra.io.read_sbml_model("ArabidopsisCoreModel1.xml")

# Adding cyclic electron transport to the model 
r = cobra.Reaction('CyclicE')
r.name = 'Cyclic_e'
r.add_metabolites({model.metabolites.get_by_id('Fdrd_h'): -2})
r.add_metabolites({model.metabolites.get_by_id('PQ_h'): -1})
r.add_metabolites({model.metabolites.get_by_id('H_h'): -2})
r.add_metabolites({model.metabolites.get_by_id('Fdox_h'): +2})
r.add_metabolites({model.metabolites.get_by_id('PQH2_h'): +1})
r.subsystem = 'light reactions'
model.add_reaction(r)
print(r.reaction)
print("Added cyclic electron transport to the model \n")

# Changing the directionality of Fum_c to Mal_c to go the other way (day-time model) 
r = model.reactions.get_by_id('FumHA_c')
r.add_metabolites({model.metabolites.get_by_id('Fum_c'): +2})
r.add_metabolites({model.metabolites.get_by_id('H2O_c'): +2})
r.add_metabolites({model.metabolites.get_by_id('Mal_c'): -2})
print(r.reaction)
print(r.bounds)
print("Directionality changed. Dyson et al. 2016 \n")

# # Creating a Malate "Storage" Reactions
# r = cobra.Reaction('Mal_Store')
# r.name = 'Mal_Store'
# r.add_metabolites({model.metabolites.get_by_id('Mal_c'): -1})
# model.add_reaction(r)
# print(r.reaction)

# Creating a Fumarate "Storage" Reactions
r = cobra.Reaction('Fum_Store')
r.name = 'Fum_Store'
r.add_metabolites({model.metabolites.get_by_id('Fum_c'): -1})
model.add_reaction(r)
print(r.reaction)

# Creating a Starch "Storage" Reactions
r = cobra.Reaction('Starch_Store')
r.name = 'Starch_Store'
r.add_metabolites({model.metabolites.get_by_id('starch1_h'): -1})
model.add_reaction(r)
print(r.reaction)
# Deleting other Starch export reactions
r = model.reactions.get_by_id('Ex_starch')
r.lower_bound = 0.0
r.upper_bound = 0.0

# Setting Directionality of Export Reactions
r = model.reactions.get_by_id('Tr_GPT1')
print(r.reaction)
r.lower_bound = -1000.0
r.upper_bound = 1000.0
print(r.reaction)
print(r.bounds)
print("Set to be bidirectional. Dyson et al. 2015 \n")

# Setting Directionality of Export Reactions
r = model.reactions.get_by_id('Tr_PPT')
print(r.reaction)
r.lower_bound = 0.0
r.upper_bound = 0.0
print(r.reaction)
print(r.bounds)
print("Testing if this eliminates TPT1 loop.")

# Deleting the other TP export option
r = model.reactions.get_by_id('Tr_TPT3')
print(r.reaction)
r.lower_bound = 0.0
r.upper_bound = 0.0
print(r.reaction)
print(r.bounds)
print("Testing if this eliminates TPT1 loop.")

for rname in ['Tr_DIT2','Tr_DIT1']:
    r = model.reactions.get_by_id(rname)
    print(r.reaction)
    r.lower_bound = 0.0
    r.upper_bound = 0.0
    print(r.bounds)
    print("Testing if this breaks the model.")
    

for rname in ["Tr_TPT1","Tr_TPT2"]:
    r = model.reactions.get_by_id(rname)
    print(r.reaction)
    r.lower_bound = 0.0
    r.upper_bound = 1000.0
    print(r.bounds)
    print("Set as one directional to avoid futile cycles")
    
for r in model.reactions:
    if r.subsystem == "export":
        if r.id not in ["Ex_Suc","Ex_O2"]:
            r.upper_bound = 0.0
            r.lower_bound = 0.0
print("Removed all export except Suc")

for m in ["starch2_h","starch3_h","starch2_c"]:
    model.remove_metabolites(model.metabolites.get_by_id(m))

cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00020 and ATCG00680 and ATCG00280 and ATCG00270 and ATCG00580 and ATCG00570 and ATCG00710 and ATCG00080 and ATCG00550 and ATCG00070 and ATCG00560 and ATCG00220 and ATCG00700 and (AT5G66570 or AT3G50820) and AT1G06680 and (AT4G21280 or AT4G05180) and AT1G79040 and AT1G44575 and ATCG00690 and AT3G21055 and AT2G30570 and AT2G06520 and AT1G67740 and ATCG00300)' for <Reaction PSII_h at 0x7f3153e8ab10>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00540 and ATCG00720 and AT4G03280 and ATCG00730 and ATCG00600 and ATCG00590 and AT2G26500 and ATCG00210)' for <Reaction Cytb6f_h at 0x7f311ddb6f10>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '8*(ATCG00490) and 8*(AT5G38430 or AT5G38420 or AT1G67090 or AT5G38410)' for <Reaction RBC_h at 0x7f311744e350>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '4*(AT3G26650 or AT1G12900) or (2*(AT3G26650 or 

2 Fdrd_h + 2 H_h + PQ_h --> 2 Fdox_h + PQH2_h
Added cyclic electron transport to the model 

Mal_c --> Fum_c + H2O_c
(0.0, 1000.0)
Directionality changed. Dyson et al. 2016 

Fum_c --> 
starch1_h --> 
G6P_c + Pi_h --> G6P_h + Pi_c
G6P_c + Pi_h <=> G6P_h + Pi_c
(-1000.0, 1000.0)
Set to be bidirectional. Dyson et al. 2015 

PEP_c + Pi_h --> PEP_h + Pi_c
PEP_c + Pi_h --> PEP_h + Pi_c
(0.0, 0.0)
Testing if this eliminates TPT1 loop.
DHAP_h + Pi_c <=> DHAP_c + Pi_h
DHAP_h + Pi_c --> DHAP_c + Pi_h
(0.0, 0.0)
Testing if this eliminates TPT1 loop.
Glu_h + Mal_c --> Glu_c + Mal_h
(0.0, 0.0)
Testing if this breaks the model.
KG_c + Mal_h --> KG_h + Mal_c
(0.0, 0.0)
Testing if this breaks the model.
PGA_h + Pi_c <=> PGA_c + Pi_h
(0.0, 1000.0)
Set as one directional to avoid futile cycles
GAP_h + Pi_c <=> GAP_c + Pi_h
(0.0, 1000.0)
Set as one directional to avoid futile cycles
Removed all export except Suc


In [2]:
rem_names = ["CO2_c","CO2_h","CO2_m","ATP_c","ATP_h","ATP_m","ADP_c","ADP_h","ADP_m",
             "NADP_c","NADP_h","NADP_m","NADPH_c","NADPH_h","NADPH_m",
             "NAD_c","NAD_h","NAD_m","NAD_p","NADH_c","NADH_h","NADH_m","NADH_p",
             "cellulose1_c","cellulose2_c","cellulose3_c",
             "CoA_m","CoA_c","CoA_h",
             # Remove conversion steps 
             "M_DASH_CoA_h","A_DASH_CoA_m","A_DASH_CoA_c","A_DASH_CoA_h","P_DASH_HPR_h","H_DASH_Eth_DASH_ThPP_m",
             "A_DASH_DHL_m","S_DASH_CoA_m","M_DASH_THF_m","M_DASH_ACP_h","F_DASH_THF_h","A_DASH_Glu_h",
             "A_DASH_GluP_h","H_DASH_Ser_h","PH_DASH_Ser_h","H_DASH_Cys_h","M_DASH_THF_c","5M_DASH_THF_c",
             "H_DASH_Cys_c","A_DASH_Glu_DASH_SeA_h","aH_DASH_Cys_c","PR_DASH_ATP_h","PR_DASH_AMP_h",
             "P_DASH_AICAR_DASH_P_h","Pu_DASH_AICAR_DASH_P_h","IA_DASH_P_h","Hisol_DASH_P_h","Asp_DASH_SeA_h",
             "H_DASH_Eth_DASH_ThPP_h","Glu_DASH_SeA_m","Glu_DASH_SeA_c","Glu_DASH_SeA_h","A_DASH_Orn_h",
             "PR_DASH_ANT_h","CPD_DASH_Ru5P_h","DC_DASH_AMP_h","Ind_DASH_GP_h","Arg_DASH_SCA_h","A_DASH_Ser_c",
             "A_DASH_Ser_h","SCA_DASH_SeA_m","CB_DASH_Asp_h","A_DASH_Ser_m","S_DASH_DHL_m",
             # Remove amino acids 
             "Asp_h","Asp_h","Arg_h","Arg_h","Asp_c","Ala_p","Asp_p","Asp_m","Ala_m","Ala_h",
             "Ala_c","Arg_c","Lys_c","Lys_m","His_m","His_c","DHO_c","Arg_p","Asn_p","Cys_p","Gln_p","His_p","Ile_c",
             "Ile_p","Leu_c","Leu_p","Lys_p","Met_p","Phe_c","Phe_p","Pro_p","Thr_p","Trp_c","Trp_p","Tyr_c","Tyr_p",
             "Val_c","Val_p","Asn_h","Asn_m","Ile_m","Leu_m","Met_h","Met_m","Phe_m","Pro_h","Thr_m","Trp_m","Tyr_m",
             "Val_m","Glu_c","Glu_h","Glu_m","Glu_p","Gln_c","Gln_h","Gln_m","Gln_p","Gly_c","Gly_h","Gly_m","Gly_p"]

In [3]:
# Adding all of the nodes to the graph 
G = nx.DiGraph() 
saves = []
CM = []
for m in model.metabolites:
    # Adding only the carbon backbone
    if "C"  in m.formula:
        save_all = []
        save_all.append(m.id)
        save_all.append(m.name)
        if m.id not in rem_names:
            G.add_node(m.id)
            save_all.append("Yes")
            CM.append(m.id)
        else:
            save_all.append("No")
        saves.append(save_all)

# Saving The Nodes To File 
workbook = xlsxwriter.Workbook('GraphMetabos.xlsx')
worksheet = workbook.add_worksheet()
worksheet.write(0, 0,"MetaboliteID")
worksheet.write(0, 1,"MetaboliteName")
worksheet.write(0, 2,"Included")
row = 1
col = 0
for i1, i2, i3 in saves:
    worksheet.write(row, col,     i1)
    worksheet.write(row, col + 1, i2)
    worksheet.write(row, col + 2, i3)
    row += 1
workbook.close()
# Adding the edges 
saves = []
for r in model.reactions:
    save_all = []
    save_all.append(r.id)
    save_all.append(str(r.reaction))
    prods = []
    reacs = []
    for p in r.products:
        if "C"  in p.formula:
            if p.id not in rem_names:
                prods.append(p.id)
    for s in r.reactants:
        if "C"  in s.formula:
            if s.id not in rem_names:
                reacs.append(s.id)
    if len(list(itertools.product(reacs, prods))) is not 0:
        if r.lower_bound == 0.0:
            # Forward Reactions
            G.add_edges_from(list(itertools.product(reacs, prods)),weight=1)
        if r.lower_bound < 0.0:
            # Reversible Reactions
            G.add_edges_from(list(itertools.product(reacs, prods)),weight=1)
            G.add_edges_from(list(itertools.product(prods, reacs)),weight=1)
    else: pass
    saves.append(save_all)

# Saving The Reactions To File 
workbook = xlsxwriter.Workbook('GraphReactions.xlsx')
worksheet = workbook.add_worksheet()
worksheet.write(0, 0,"ReactionID")
worksheet.write(0, 1,"Reaction")
row = 1
col = 0
for i1, i2 in saves:
    worksheet.write(row, col,     i1)
    worksheet.write(row, col + 1, i2)
    row += 1
workbook.close()

In [30]:
# Finding the longest shortest path from RuBP_h
# path_lens = []
# for nod in CM: 
#     try:
#         p = nx.all_shortest_paths(G, "RuBP_h", nod)
#         P_len = len(list(p)[0])
#         path_lens.append(P_len)
#     except: pass
# print(max(path_lens))
max_len = 14.0

# Calulating the risk factor based on Probability of Failure (P) and Severity (S)
risk_dict = {}
for nod in CM:
    S1 = G.degree[nod]
    bw_centrality = nx.betweenness_centrality(G, normalized=True)
    S2 = bw_centrality[nod]
    S = S1*S2
    try:
        p = nx.all_shortest_paths(G, "RuBP_h", nod)
        paths = list(p)
        P_no = len(paths) # Number of shortest paths 
        P_len = len(paths[0])/max_len # Shortest path length / longest shortest path length
        P = P_len/P_no
        risk_dict[nod]=S*P
        if (S*P) > 0.21: #4:
            print("{} ({}), Risk factor {}".format(model.metabolites.get_by_id(nod).name,nod,S*P))
#             for ns in G.neighbors(nod):
#                 print(ns)
        if model.metabolites.get_by_id(nod).id == "Fum_c":
            print(S*P)
    except:
        #print("No path to node {} exists.".format(nod))
        P = 0
    

Glyceraldehyde 3-phosphate (GAP_h), Risk factor 0.582306593402
Fructose 6-phosphate (F6P_h), Risk factor 0.293659096379
Ribose 5-phosphate (R5P_h), Risk factor 0.313714979335
Glucose 6-phosphate (G6P_h), Risk factor 0.218001903431
Glucose 6-phosphate (G6P_c), Risk factor 0.312663434211
Malate (Mal_c), Risk factor 0.336884554947
Oxaloacetate (OAA_c), Risk factor 0.435109128125
Phosphoenolpyruvate (PEP_h), Risk factor 0.257992618377
Pyruvate (Pyr_h), Risk factor 1.02770342586
Adenosine 5-phosphate (AMP_h), Risk factor 0.212877037036
alpha-Ketoglutarate, 2-Oxoglutarate (KG_h), Risk factor 0.753472384267
0.0


In [26]:
min(risk_dict.values())
np.mean(risk_dict.values())

0.05055949506563362